# <b>Beam Sarch Exploration</b>

### <b>Greedy Decoder</b>. Picks the word with <b>highest probability</b> from a vocabulary

In [10]:
import numpy as np
from numpy import array, argmax

# greedy decoder
def greedy_decoder(data):
    # index for largest probability each row
    return [argmax(s) for s in data]


#### Example: assume "data" is the result of a prediction task that predicts the next 10 words given the previous word over a vocab of 5 words.

I start from a prediciton that tolds me that word1 is probable at 0.1, word2 at 0.2, ..., word3 at 0.5

The greedy decoder returns the vocabulary entries (indexes pointing to vocabulary) with highest probability for each word

In [93]:
vocab = {0:'pippo', 1:'sandrino', 2:'pieroni', 3:'marzullo', 4:'catorcio'}

In [94]:
#np.random.uniform(0, 1, 5)

In [95]:
def random_data(sequence_size, vocab_size):
    result = []
    for i in range(0, sequence_size):
        result.append(np.random.uniform(0, 1, vocab_size))
    return array(result)

In [108]:
data = random_data(10,5)

print(data)

[[ 0.12652083  0.22813471  0.2987071   0.34933662  0.42458194]
 [ 0.42307503  0.62971268  0.47829648  0.41690849  0.55282322]
 [ 0.30335729  0.85727797  0.50610382  0.03081907  0.38423528]
 [ 0.76025498  0.26909101  0.82494793  0.00325186  0.69590617]
 [ 0.76407061  0.05971319  0.70694349  0.30355835  0.32372128]
 [ 0.62742345  0.00705829  0.64530333  0.07901538  0.15588304]
 [ 0.01080289  0.29544724  0.5133111   0.16263049  0.68947834]
 [ 0.32025736  0.90762189  0.588747    0.78900124  0.97637995]
 [ 0.90069263  0.5889007   0.07500405  0.14331263  0.05722853]
 [ 0.34206635  0.94817187  0.30743983  0.1173878   0.84562785]]


In [121]:
# decode sequence
result = greedy_decoder(data)

print(result)

[4, 3, 0, 3, 4, 0, 4, 1, 0, 3]


In [122]:
# decode highest probability word indexes to vocab words
def decode_from_vocab(vocab, idx_array):
    decoded = []
    for idx in idx_array:
        decoded.append(vocab[idx])
    return decoded

In [123]:
decode_from_vocab(vocab, result)

['catorcio',
 'marzullo',
 'pippo',
 'marzullo',
 'catorcio',
 'pippo',
 'catorcio',
 'sandrino',
 'pippo',
 'marzullo']

### <b>Beam Search Decoder</b>. Expands all possible next steps and keeps the <b>k most likely</b> (where k controls the number of beams or parallel searches through the sequence of probabilities)


*The local beam search algorithm keeps track of k states rather than just one. It begins with k randomly generated states. At each step, all the successors of all k states are generated. If any one is a goal, the algorithm halts. Otherwise, it selects the k best successors from the complete list and repeats.*

Common beam width values are 1 for a greedy search and values of 5 or 10 for common benchmark problems in machine translation

In [137]:
from math import log

# beam search
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    # walk over each step in sequence
    for row in data:
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            print("seq:", seq, "score:", score, "")
            for j in range(len(row)):
                candidate = [seq + [j], score * -log(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[:k]
    return sequences

In [138]:
data = random_data(10,5)
print(data)

[[ 0.36811372  0.64113261  0.36029716  0.93636979  0.30852459]
 [ 0.59053945  0.84450863  0.17295846  0.79029918  0.70441549]
 [ 0.68071501  0.25385581  0.60166938  0.4405493   0.41234752]
 [ 0.91528242  0.21527672  0.07338004  0.01712999  0.00831657]
 [ 0.74412121  0.1433341   0.70566187  0.18815439  0.01064368]
 [ 0.79480572  0.40043059  0.70812524  0.09571005  0.3651266 ]
 [ 0.21991803  0.1756754   0.75151163  0.71206413  0.61299718]
 [ 0.75346833  0.97923533  0.28098394  0.72734006  0.50163434]
 [ 0.3084521   0.82976847  0.89205979  0.8753751   0.11559779]
 [ 0.27017807  0.54588732  0.38265614  0.25468077  0.09792002]]


In [139]:
# decode sequence
beam_result = beam_search_decoder(data, 3)
# print result
for seq in beam_result:
    print(seq)
    print(decode_from_vocab(vocab, seq[0]))

[] 1.0
[3] 0.06574480480467146
[1] 0.4445189587972926
[0] 0.9993633660013567
[3, 1] 0.011110892945031363
[3, 3] 0.015472625264490938
[3, 4] 0.023036119013964466
[3, 1, 0] 0.004273377657532197
[3, 1, 2] 0.005644857915587794
[3, 3, 0] 0.005950950246371683
[3, 1, 0, 0] 0.000378290534883231
[3, 1, 2, 0] 0.0004996975440407722
[3, 3, 0, 0] 0.0005267936354268913
[3, 1, 0, 0, 0] 0.00011180427698523896
[3, 1, 0, 0, 2] 0.00013187930525892345
[3, 1, 2, 0, 0] 0.00014768628202665266
[3, 1, 0, 0, 0, 0] 2.5676698601425663e-05
[3, 1, 0, 0, 2, 0] 3.028708081843646e-05
[3, 1, 2, 0, 0, 0] 3.391727269668014e-05
[3, 1, 0, 0, 0, 0, 2] 7.3350265460233015e-06
[3, 1, 0, 0, 2, 0, 2] 8.652067980127696e-06
[3, 1, 0, 0, 0, 0, 3] 8.71948081699294e-06
[3, 1, 0, 0, 0, 0, 2, 1] 1.5391293566720985e-07
[3, 1, 0, 0, 2, 0, 2, 1] 1.8154878841381771e-07
[3, 1, 0, 0, 0, 0, 3, 1] 1.829633310277377e-07
[3, 1, 0, 0, 0, 0, 2, 1, 2] 1.7580262108084776e-08
[3, 1, 0, 0, 0, 0, 2, 1, 3] 2.0486241834258827e-08
[3, 1, 0, 0, 2, 0, 2, 1,